In [7]:
## Para empezar a ejecutar el codigo necesita un dataset ESOL (Delaney) el cual puede descargar en su sitio web ##

import pandas as pd

# Cambia el nombre si tu archivo es diferente
df = pd.read_csv("delaney-processed.csv")
print(df.columns)  # Para ver los nombres de las columnas

Index(['Compound ID', 'ESOL predicted log solubility in mols per litre',
       'Minimum Degree', 'Molecular Weight', 'Number of H-Bond Donors',
       'Number of Rings', 'Number of Rotatable Bonds', 'Polar Surface Area',
       'measured log solubility in mols per litre', 'smiles'],
      dtype='object')


In [8]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import joblib

# 1. Cargar el dataset ESOL
df = pd.read_csv('delaney-processed.csv')

# 2. Seleccionar descriptores y variable objetivo
X = df[['Molecular Weight', 'Number of H-Bond Donors']]
y = df['measured log solubility in mols per litre']

# 3. Entrenar el modelo Random Forest
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=42)
model.fit(X_train, y_train)
print("R² test:", model.score(X_test, y_test))  # Evaluación del modelo

# 4. Guardar el modelo entrenado
joblib.dump(model, 'chemcomp_model.pkl')

R² test: 0.6208097872365256


['chemcomp_model.pkl']

In [10]:
import pandas as pd
import joblib
from rdkit import Chem
from rdkit.Chem import Descriptors
import os

# Ruta al modelo entrenado
modelo_path = '/home/angel/Escritorio/smiles_chebi/ML/chemcomp_model.pkl'
model = joblib.load(modelo_path)

# Archivos a procesar (en la carpeta de tus compuestos)
archivos = [
    "antocianinas.csv",
    "clorofilas.csv",
    "flavonoides.csv",
    "xantofilas.csv"
]

# Ruta base donde están tus archivos de compuestos
ruta_base = '/home/angel/Escritorio/smiles_chebi/'

def get_desc(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [None, None]
    return [Descriptors.MolWt(mol), Descriptors.NumHDonors(mol)]

for archivo in archivos:
    ruta_archivo = os.path.join(ruta_base, archivo)
    df = pd.read_csv(ruta_archivo)
    desc = df['chebi_smiles'].apply(get_desc)
    desc_df = pd.DataFrame(desc.tolist(), columns=['Molecular Weight', 'Number of H-Bond Donors'])
    df_pred = pd.concat([df, desc_df], axis=1).dropna(subset=['Molecular Weight', 'Number of H-Bond Donors'])
    df_pred['solubility_pred'] = model.predict(df_pred[['Molecular Weight', 'Number of H-Bond Donors']])
    resultado = archivo.replace('.csv', '_con_solubilidad.csv')
    ruta_resultado = os.path.join(ruta_base, resultado)
    df_pred.to_csv(ruta_resultado, index=False)
    print(f"¡Predicciones guardadas en {ruta_resultado}!")

¡Predicciones guardadas en /home/angel/Escritorio/smiles_chebi/antocianinas_con_solubilidad.csv!
¡Predicciones guardadas en /home/angel/Escritorio/smiles_chebi/clorofilas_con_solubilidad.csv!
¡Predicciones guardadas en /home/angel/Escritorio/smiles_chebi/flavonoides_con_solubilidad.csv!
¡Predicciones guardadas en /home/angel/Escritorio/smiles_chebi/xantofilas_con_solubilidad.csv!
